<a href="https://colab.research.google.com/github/amod404/Deep-Learning-Notes/blob/master/keras-tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('diabetes.csv')

In [8]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [9]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [10]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [11]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [14]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [67]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [16]:
model = Sequential()
model.add(Dense(32,activation='relu', input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - accuracy: 0.4671 - loss: 0.7476 - val_accuracy: 0.6234 - val_loss: 0.6497
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6107 - loss: 0.6557 - val_accuracy: 0.7403 - val_loss: 0.5964
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7203 - loss: 0.5969 - val_accuracy: 0.7922 - val_loss: 0.5609
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7603 - loss: 0.5580 - val_accuracy: 0.7922 - val_loss: 0.5354
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7636 - loss: 0.5312 - val_accuracy: 0.7987 - val_loss: 0.5172
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7605 - loss: 0.5467 - val_accuracy: 0.7987 - val_loss: 0.5064
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7518 - loss: 0.5143 - val_accuracy: 0.8052 - val_loss: 0.4964
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7771 - loss: 0.4958 - val_accuracy: 0.8052 - val_loss

In [18]:
#1. How to select appropriate optimizer
#2. No of nodes in a layer
#3. No of hidden layers
#4. All in one model

In [19]:
pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 13.0 MB/s eta 0:00:00


In [20]:
import kerastuner as kt

/tmp/ipython-input-20-1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [21]:
from re import M
def build_model(hp):

    model = Sequential()
    model.add(Dense(32,activation='relu', input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    optimizers = hp.Choice('optimizer', ['adam','sgd','rmsprop','adadelta'])

    model.compile(optimizer=optimizers, loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [22]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        overwrite=False)

In [23]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test),verbose=1 )

Trial 4 Complete [00h 00m 09s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 22s


In [24]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7922077775001526

Trial 1 summary
Hyperparameters:
optimizer: adam
Score: 0.7597402334213257

Trial 2 summary
Hyperparameters:
optimizer: sgd
Score: 0.6818181872367859

Trial 0 summary
Hyperparameters:
optimizer: adadelta
Score: 0.37012988328933716


In [25]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [26]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [27]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 162ms/step - accuracy: 0.7574 - loss: 0.5190 - val_accuracy: 0.8052 - val_loss: 0.4927
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7594 - loss: 0.5007 - val_accuracy: 0.7922 - val_loss: 0.4816
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7704 - loss: 0.4938 - val_accuracy: 0.7922 - val_loss: 0.4760
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7778 - loss: 0.4797 - val_accuracy: 0.7922 - val_loss: 0.4714
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7919 - loss: 0.4741 - val_accuracy: 0.7987 - val_loss: 0.4694
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7642 - loss: 0.4782 - val_accuracy: 0.7922 - val_loss: 0.4676
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7754 - loss: 0.4649 - val_accuracy: 0.7857 - val_loss: 0.4670
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7513 - loss: 0.4992 - val_accuracy: 0.7922 -

tuning the number of nodes in a layer

In [32]:
def build_model(hp):

  model = Sequential()
  units = hp.Int('units',8,128, step=8)

  model.add(Dense(units=units, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])\

  return model


In [40]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=15,
                        directory='mydir',
                        project_name='proj')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [41]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 15 Complete [00h 00m 06s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 58s


In [42]:
tuner.results_summary()

Results summary
Results in mydir/proj
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 00 summary
Hyperparameters:
units: 96
Score: 0.8116883039474487

Trial 10 summary
Hyperparameters:
units: 48
Score: 0.8116883039474487

Trial 02 summary
Hyperparameters:
units: 104
Score: 0.8051947951316833

Trial 08 summary
Hyperparameters:
units: 56
Score: 0.798701286315918

Trial 01 summary
Hyperparameters:
units: 112
Score: 0.7922077775001526

Trial 11 summary
Hyperparameters:
units: 120
Score: 0.7857142686843872

Trial 13 summary
Hyperparameters:
units: 64
Score: 0.7857142686843872

Trial 14 summary
Hyperparameters:
units: 128
Score: 0.7857142686843872

Trial 03 summary
Hyperparameters:
units: 88
Score: 0.7792207598686218

Trial 09 summary
Hyperparameters:
units: 80
Score: 0.7792207598686218


In [43]:
tuner.get_best_hyperparameters()[0].values

{'units': 96}

In [44]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [45]:
model.fit(X_train, y_train, batch_size=32,epochs=100)

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.8055 - loss: 0.4643
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7540 - loss: 0.4876 
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7637 - loss: 0.4538 
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7906 - loss: 0.4447 
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7725 - loss: 0.4482 
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7845 - loss: 0.4360 
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7835 - loss: 0.4518 
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7741 - loss: 0.4324
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7864 - loss: 0.4340 
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7818 - loss: 0.4250 
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7829 - loss: 0.4238 
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step -

In [51]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(72, activation='relu',input_dim=8))

  for i in range(hp.Int('num_layer', min_value=1, max_value=10)):

    model.add(Dense(96,activation='relu'))

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

  return model


In [52]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=10,
                        directory='mydir',
                        project_name='proj2')

Reloading Tuner from mydir/proj2/tuner0.json


In [53]:
tuner.search(X_train, y_train,batch_size=32 ,epochs=5, validation_data=(X_test, y_test) )

Trial 10 Complete [00h 00m 06s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 02m 36s


In [54]:
tuner.results_summary()

Results summary
Results in mydir/proj2
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 01 summary
Hyperparameters:
num_layer: 2
Score: 0.8246753215789795

Trial 02 summary
Hyperparameters:
num_layer: 9
Score: 0.8181818127632141

Trial 00 summary
Hyperparameters:
num_layer: 6
Score: 0.8181818127632141

Trial 05 summary
Hyperparameters:
num_layer: 4
Score: 0.8181818127632141

Trial 04 summary
Hyperparameters:
num_layer: 1
Score: 0.8116883039474487

Trial 03 summary
Hyperparameters:
num_layer: 7
Score: 0.798701286315918

Trial 09 summary
Hyperparameters:
num_layer: 8
Score: 0.798701286315918

Trial 06 summary
Hyperparameters:
num_layer: 5
Score: 0.7922077775001526

Trial 07 summary
Hyperparameters:
num_layer: 10
Score: 0.7922077775001526

Trial 08 summary
Hyperparameters:
num_layer: 3
Score: 0.7922077775001526


Last code, adding all hyperparametertuning


In [68]:
def build_model(hp):
  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

    if counter == 0:
      model.add(
          Dense(
              hp.Int('units'+str(i), min_value=8, max_value=128, step=8),
              activation= hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']),
              input_dim=8
              )
           )
      model.add(Dropout(hp.Choice('dropout'+ str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    else:
      model.add(
          Dense(
              hp.Int('units'+str(i), min_value=8, max_value=128, step=8),
              activation= hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])
              )
           )
      model.add(Dropout(hp.Choice('dropout'+ str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    counter+=1

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop','adam','sgd','adadelta']),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model


In [72]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=10,
                        directory='mydir',
                        project_name='final2')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [73]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 10 Complete [00h 00m 14s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 02m 39s


In [74]:
tuner.results_summary()

Results summary
Results in mydir/final2
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 00 summary
Hyperparameters:
num_layers: 7
units0: 112
activation0: tanh
dropout0: 0.7
optimizer: sgd
units1: 8
activation1: relu
dropout1: 0.1
units2: 8
activation2: relu
dropout2: 0.1
units3: 8
activation3: relu
dropout3: 0.1
units4: 8
activation4: relu
dropout4: 0.1
units5: 8
activation5: relu
dropout5: 0.1
units6: 8
activation6: relu
dropout6: 0.1
Score: 0.6428571343421936

Trial 01 summary
Hyperparameters:
num_layers: 5
units0: 24
activation0: sigmoid
dropout0: 0.5
optimizer: adam
units1: 24
activation1: relu
dropout1: 0.5
units2: 120
activation2: tanh
dropout2: 0.5
units3: 80
activation3: relu
dropout3: 0.9
units4: 96
activation4: relu
dropout4: 0.3
units5: 32
activation5: sigmoid
dropout5: 0.7
units6: 64
activation6: tanh
dropout6: 0.1
Score: 0.6428571343421936

Trial 02 summary
Hyperparameters:
num_layers: 4
units0: 16
activation0: relu
dropout0: 0.2
optimizer: a

In [75]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7,
 'units0': 112,
 'activation0': 'tanh',
 'dropout0': 0.7,
 'optimizer': 'sgd',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 8,
 'activation5': 'relu',
 'dropout5': 0.1,
 'units6': 8,
 'activation6': 'relu',
 'dropout6': 0.1}

In [76]:
model = tuner.get_best_models(num_models=1)[0]

In [77]:
model.fit(X_train,y_train,epochs=200,validation_data=(X_test, y_test))

Epoch 1/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 155ms/step - accuracy: 0.6276 - loss: 0.6881 - val_accuracy: 0.6429 - val_loss: 0.6840
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.6573 - loss: 0.6815 - val_accuracy: 0.6429 - val_loss: 0.6793
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6445 - loss: 0.6779 - val_accuracy: 0.6429 - val_loss: 0.6754
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6097 - loss: 0.6806 - val_accuracy: 0.6429 - val_loss: 0.6714
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6608 - loss: 0.6653 - val_accuracy: 0.6429 - val_loss: 0.6678
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6423 - loss: 0.6685 - val_accuracy: 0.6429 - val_loss: 0.6652
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6692 - loss: 0.6577 - val_accuracy: 0.6429 - val_loss: 0.6629
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6473 - loss: 0.6585 - val_accuracy: 0.6429 